<a href="https://colab.research.google.com/github/Dotunbey/Branham-ai/blob/main/Find_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:

# @title 1. Robust Installation (Fixes xformers build error)
import torch
import sys
from packaging.version import Version as V

# 1. Check if GPU is actually active
if not torch.cuda.is_available():
    raise RuntimeError("⚠️ RUNTIME ERROR: You are on CPU. Go to Runtime > Change runtime type > T4 GPU")

print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
print(f"🔍 PyTorch Version: {torch.__version__}")

# 2. Smart-Select the correct xformers version to avoid compiling
# Colab recently updated PyTorch, so we need to match the version exactly.
major_version = V(torch.__version__)
if major_version < V("2.4.0"):
    xformers_version = "xformers<0.0.27" # For older Colab runtimes
else:
    xformers_version = "xformers"        # For newest Colab (PyTorch 2.4+)

print(f"📦 Selecting {xformers_version} to match PyTorch...")

# 3. Install Unsloth and Dependencies
# We use --no-deps for the libraries to prevent them from fighting each other
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --no-cache-dir
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes {xformers_version}

print("\n✅ INSTALLATION SUCCESSFUL. You can now run the training script.")

✅ GPU Detected: Tesla T4
🔍 PyTorch Version: 2.9.0+cu126
📦 Selecting xformers to match PyTorch...
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-17fjl6ge/unsloth_dca68ab18e794bab82cfa4f530c927e9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-17fjl6ge/unsloth_dca68ab18e794bab82cfa4f530c927e9
  Resolved https://github.com/unslothai/unsloth.git to commit 872e191494104d177d854a2e2b430a03a193d4e2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully uninstalled trl-0.24.0

✅ INSTALLATION SUCCESSFUL. You can now run the tr

In [ ]:

from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# CONFIGURATION
DATASET_PATH = "/content/drive/My Drive/Branham_Sermons/branham_completion_data.jsonl"
OUTPUT_DIR = "branham_outputs"

# LOAD MODEL
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# ADD ADAPTERS
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Adjusted for stability
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# LOAD DATA
dataset = load_dataset("json", data_files = {"train": DATASET_PATH}, split = "train")

# TRAIN
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = OUTPUT_DIR,
    ),
)

trainer.train()

# EXPORT
print("💾 Saving Model...")
model.save_pretrained_gguf("Branham_Voice_Model", tokenizer, quantization_method = "q4_k_m")
print(f"✅ DONE. Check the 'Branham_Voice_Model' folder.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [03:59<00:00, 59.90s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [04:08<00:00, 62.16s/it]


Unsloth: Merge process complete. Saved to `/content/Branham_Voice_Model`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
